# Knowledge Graph Reasoning with TensorLogic

This notebook demonstrates multi-hop reasoning over knowledge graphs using TensorLogic. We'll implement complex queries like grandparent and aunt/uncle inference using tensor operations.

## What You'll Learn

1. Building a family knowledge graph
2. Multi-hop reasoning (grandparent inference)
3. Rule composition (aunt/uncle inference)
4. Implication checking

In [ ]:
from tensorlogic import (
    create_backend,
    logical_and,
    logical_or,
    logical_implies,
    exists,
    forall,
    quantify,
    reason,
)
import numpy as np

backend = create_backend()
print(f"Using backend: {type(backend).__name__}")

## Family Knowledge Graph

We'll create a 3-generation family tree:

```
Generation 1: Alice & Bob (married)
                |          |
        +-------+          +-------+
        |                          |
Generation 2: Carol (+ David)     Eve
                |                  |
        +-------+-------+          |
        |               |          |
Generation 3: Frank   Grace     Henry
```

In [ ]:
# Define entities (8 people in 3 generations)
entities = ["Alice", "Bob", "Carol", "David", "Eve", "Frank", "Grace", "Henry"]
n = len(entities)
entity_to_idx = {name: i for i, name in enumerate(entities)}

print("Family Members:")
for name, idx in entity_to_idx.items():
    print(f"  [{idx}] {name}")

In [ ]:
# Create Parent relation tensor (8x8 matrix)
# Parent[i, j] = 1.0 means entity i is parent of entity j
parent = np.zeros((n, n), dtype=np.float32)
parent_facts = [
    ("Alice", "Carol"), ("Alice", "Eve"),
    ("Bob", "Carol"), ("Bob", "Eve"),
    ("Carol", "Frank"), ("Carol", "Grace"),
    ("David", "Frank"), ("David", "Grace"),
    ("Eve", "Henry"),
]
for p, c in parent_facts:
    parent[entity_to_idx[p], entity_to_idx[c]] = 1.0

print("Parent Relations:")
for p, c in parent_facts:
    print(f"  {p} -> {c}")

In [ ]:
# Create Sibling relation tensor (symmetric)
sibling = np.zeros((n, n), dtype=np.float32)
sibling_pairs = [("Carol", "Eve"), ("Frank", "Grace")]
for s1, s2 in sibling_pairs:
    sibling[entity_to_idx[s1], entity_to_idx[s2]] = 1.0
    sibling[entity_to_idx[s2], entity_to_idx[s1]] = 1.0

print("Sibling Relations (symmetric):")
for s1, s2 in sibling_pairs:
    print(f"  {s1} <-> {s2}")

In [ ]:
# Create Loves relation tensor (soft values)
loves = np.zeros((n, n), dtype=np.float32)
# Parents love their children
for p, c in parent_facts:
    loves[entity_to_idx[p], entity_to_idx[c]] = 0.95
# Siblings have affection
for s1, s2 in sibling_pairs:
    loves[entity_to_idx[s1], entity_to_idx[s2]] = 0.7
    loves[entity_to_idx[s2], entity_to_idx[s1]] = 0.7

print("Loves Relation (sample):")
print(f"  Alice loves Carol: {loves[entity_to_idx['Alice'], entity_to_idx['Carol']]:.2f}")
print(f"  Carol loves Eve: {loves[entity_to_idx['Carol'], entity_to_idx['Eve']]:.2f}")

## Grandparent Inference

**Rule**: `Grandparent(x,z) <- exists y: Parent(x,y) AND Parent(y,z)`

This rule says: x is a grandparent of z if there exists an intermediate person y such that x is parent of y AND y is parent of z.

**Key Insight**: This is equivalent to matrix multiplication!

In [ ]:
# Grandparent = Parent @ Parent
# Matrix multiplication computes: sum_y (Parent[x,y] * Parent[y,z])
grandparent_paths = np.matmul(parent, parent)

# Convert to boolean (1.0 if any path exists)
grandparent = (grandparent_paths > 0).astype(np.float32)

print("Grandparent Inference: Grandparent(x,z) <- exists y: Parent(x,y) AND Parent(y,z)")
print("\nInferred Grandparent Relations:")
for i, gp_name in enumerate(entities):
    for j, gc_name in enumerate(entities):
        if grandparent[i, j] > 0:
            # Find the intermediate parent(s)
            for k, p_name in enumerate(entities):
                if parent[i, k] > 0 and parent[k, j] > 0:
                    print(f"  {gp_name} -> {gc_name} (via {p_name})")

## Aunt/Uncle Inference

**Rule**: `AuntUncle(x,z) <- exists y: Sibling(x,y) AND Parent(y,z)`

This rule says: x is an aunt/uncle of z if there exists y such that x is a sibling of y AND y is parent of z.

In [ ]:
# AuntUncle = Sibling @ Parent
aunt_uncle = (np.matmul(sibling, parent) > 0).astype(np.float32)

print("Aunt/Uncle Inference: AuntUncle(x,z) <- exists y: Sibling(x,y) AND Parent(y,z)")
print("\nInferred Aunt/Uncle Relations:")
for i, au_name in enumerate(entities):
    for j, niece_name in enumerate(entities):
        if aunt_uncle[i, j] > 0:
            # Find the sibling who is the parent
            for k, sib_name in enumerate(entities):
                if sibling[i, k] > 0 and parent[k, j] > 0:
                    print(f"  {au_name} is aunt/uncle of {niece_name} (sibling of {sib_name})")

## Implication Checking

**Rule**: `Parent(x,y) -> Loves(x,y)`

Let's verify: Do all parents love their children?

Implication P -> Q is computed as `max(1-P, Q)`.

In [ ]:
# Check implication: Parent(x,y) -> Loves(x,y)
implication = logical_implies(parent, loves, backend=backend)

print("Implication Check: Parent(x,y) -> Loves(x,y)")
print("Formula: max(1 - Parent, Loves)")
print("\nResults for parent-child pairs:")
threshold = 0.5
for i, p_name in enumerate(entities):
    for j, c_name in enumerate(entities):
        if parent[i, j] > 0:
            imp_val = float(implication[i, j])
            status = "satisfied" if imp_val >= threshold else "violated"
            print(f"  {p_name} -> {c_name}: Parent={parent[i, j]:.1f}, Loves={loves[i, j]:.2f}, Implication={imp_val:.2f} ({status})")

## Quantified Queries

Let's use TensorLogic's quantifiers to answer more complex questions.

In [ ]:
# Query 1: Who has children? (exists y: Parent(x, y))
has_children = exists(parent, axis=1, backend=backend)

print("Query: exists y: Parent(x, y) - 'Who has children?'")
print("\nResults:")
for i, name in enumerate(entities):
    val = float(has_children[i])
    children = [entities[j] for j in range(n) if parent[i, j] > 0]
    if val > 0:
        print(f"  {name}: Yes (children: {', '.join(children)})")
    else:
        print(f"  {name}: No")

In [ ]:
# Query 2: Does everyone love all their children? (forall y: Parent(x,y) -> Loves(x,y))
implication = logical_implies(parent, loves, backend=backend)
forall_result = forall(implication, axis=1, backend=backend)

print("Query: forall y: Parent(x,y) -> Loves(x,y)")
print("'Does everyone love all their children?'")
print("\nResults (1.0 = loves all children):")
for i, name in enumerate(entities):
    val = float(forall_result[i])
    print(f"  {name}: {val:.1f}")

In [ ]:
# Query 3: Who has a married sibling?
# exists y: Sibling(x,y) AND Married(y, someone)

# First, create a Married relation
married = np.zeros((n, n), dtype=np.float32)
married_pairs = [("Alice", "Bob"), ("Carol", "David")]
for m1, m2 in married_pairs:
    married[entity_to_idx[m1], entity_to_idx[m2]] = 1.0
    married[entity_to_idx[m2], entity_to_idx[m1]] = 1.0

# Check if each person is married
is_married = (np.sum(married, axis=1) > 0).astype(np.float32)

# Sibling(x, y) AND IsMarried(y)
sibling_and_married = logical_and(sibling, is_married.reshape(1, -1), backend=backend)

# Exists over y
has_married_sibling = exists(sibling_and_married, axis=1, backend=backend)

print("Query: exists y: Sibling(x,y) AND IsMarried(y)")
print("'Who has a married sibling?'")
print("\nResults:")
for i, name in enumerate(entities):
    val = float(has_married_sibling[i])
    if val > 0:
        for j, sib_name in enumerate(entities):
            if sibling[i, j] > 0 and is_married[j] > 0:
                print(f"  {name}: Yes (married sibling: {sib_name})")
                break
    else:
        print(f"  {name}: No")

## Rule Chaining

TensorLogic allows you to chain multiple rules together. Let's compute great-grandparents.

In [ ]:
# Great-grandparent = Parent @ Parent @ Parent
great_grandparent = (np.matmul(np.matmul(parent, parent), parent) > 0).astype(np.float32)

print("Great-Grandparent Inference (3-hop):")
print("GreatGrandparent(x,w) <- exists y,z: Parent(x,y) AND Parent(y,z) AND Parent(z,w)")
print("\nInferred Great-Grandparent Relations:")

found = False
for i, ggp_name in enumerate(entities):
    for j, ggc_name in enumerate(entities):
        if great_grandparent[i, j] > 0:
            print(f"  {ggp_name} is great-grandparent of {ggc_name}")
            found = True

if not found:
    print("  (No great-grandparent relations in this 3-generation family)")

## Visualization: Relation Matrices

In [ ]:
def print_relation_matrix(relation, entities, name):
    """Print a relation matrix in a readable format."""
    print(f"\n{name} Relation Matrix:")
    print(f"{'':>8}", end="")
    for entity in entities:
        print(f"{entity[:5]:>6}", end="")
    print()
    for i, row_name in enumerate(entities):
        print(f"{row_name:>8}", end="")
        for j in range(len(entities)):
            val = relation[i, j]
            if val > 0:
                print(f"{val:>6.1f}", end="")
            else:
                print(f"{'·':>6}", end="")
        print()

print_relation_matrix(parent, entities, "Parent")
print_relation_matrix(grandparent, entities, "Grandparent (Inferred)")
print_relation_matrix(aunt_uncle, entities, "Aunt/Uncle (Inferred)")

## Summary

In this notebook, you learned:

1. **Knowledge Graph Structure**: Relations are represented as n×n tensors
2. **Multi-hop Reasoning**: Matrix multiplication computes transitive relations
3. **Rule Composition**: Complex rules combine primitive relations
4. **Implication Checking**: `max(1-P, Q)` implements logical implication
5. **Quantified Queries**: EXISTS and FORALL aggregate over entity dimensions

## Key Insight

The mathematical equivalence between logical inference and matrix operations means:
- **Grandparent** = `Parent @ Parent`
- **Aunt/Uncle** = `Sibling @ Parent`
- **Great-grandparent** = `Parent @ Parent @ Parent`

This enables GPU-accelerated batch inference over millions of entities!

## Next Steps

- **03_compilation_strategies.ipynb**: Compare different semantic interpretations
- **04_temperature_control.ipynb**: Deductive vs analogical reasoning